# Transfer Learning Part 1: Feature Extraction

Transfer learning is a leveraging a working model's existing architecture and learned paterns for our own problem.

There are two main benefits:
* Can leverage an existing neural network architecture proven to work on problems similar to our own
* Can leverage a working neural network architecture which has already learned patterns on similar data to or own, then we can adapt those patterns to our own data.

In [23]:
# Import Libraries
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Downloading and becoming one with data

In the last notebook, we used Food101 data and work with all the data however this time we will use only 10 percent of the same data and try to see power of transfer learning.

In [8]:
# Get data (%10 of 10 food classes from Food101)
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

--2023-06-21 09:20:38--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 173.194.79.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M  37.7MB/s    in 5.0s    

2023-06-21 09:20:43 (31.8 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [13]:
# Walk through 10 percent data directory and list number of files
for dir in os.walk('10_food_classes_10_percent'):
    print(f"There are {len(dir[1])} directories and {len(dir[2])} images in '{dir[0]}'")

There are 2 directories and 0 images in '10_food_classes_10_percent'
There are 10 directories and 0 images in '10_food_classes_10_percent/train'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_crea

## Creating data loaders (preparing the data)

We'll use the `ImageDataGenerator` class to load in our images in batches

In [17]:
# Set the global variables
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)

# Setup data inputs
train_dir = '10_food_classes_10_percent/train/'
test_dir = '10_food_classes_10_percent/test/'

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print('Training images:')
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SIZE,
                                               class_mode='categorical',
                                               batch_size=BATCH_SIZE)

print('\nTest images:')
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SIZE,
                                             class_mode='categorical',
                                             batch_size=BATCH_SIZE)

Training images:
Found 750 images belonging to 10 classes.

Test images:
Found 2500 images belonging to 10 classes.


## Setting up callbacks (things to run while our model trains)

Callbacks are extra functionality you can add to your models to be perfomeed during or after training. Some of the most populer callbacks:

* Tracking experiments with the TensorBoard callback
* Model checkpoint with the ModelCheckpoint callback
* Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callback

In [19]:
# Create a TensorBoard callbakc (functionized because we need to create a new one for each model)
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f'Saving TensorBoard log files to: {log_dir}')
    return tensorboard_callback

## Creating models using TensorFlow Hub

In the past notebooks, we created our model's architecture layer by layer from scratch. Now, we are going to do similar proces but majority of our model's layers will come from TensorFlow Hub. Here you can find out pretrained models: https://tfhub.dev/

After browsing in TensorFlow Hub, I found out following feature vector models:

* efficientnet: https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1
* resnet: https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5

In [22]:
# Let's compare the following models
resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'
efficientnet_url = 'https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1'

In [27]:
# Let's make a create_model() function to create a model from URL
def create_model(model_url, num_classes=10):
    '''
    Takes a TensorFlow Hub url and create a model sequential

    Args:
        model_url(str): A Tensorflow Hub url
        num_classes(int): A number of output neurons in the output layer. Should be equal to target classes. Default 10

    Returns:
        An uncomplied Keras Sequential model with model_url
    '''


    # Download the pretrained model and save it as a Keras layer
    feature_extract_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name='feature_extraction_layer',
                                           input_shape=IMAGE_SIZE+(3,)) # freeze the already learned patterns

    # Create the model
    model = tf.keras.Sequential([
        feature_extract_layer,
        layers.Dense(num_classes, activation='softmax', name='output_layer')
    ])

    return model

### Creating and testing Resnet TensorFlow Hub feature extraction model


In [30]:
# Create Resnet model
resnet_model = create_model(model_url=resnet_url, num_classes=train_data.num_classes)

In [31]:
# Summary of the resnet_model
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 2048)             23564800  
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


There are `23,585,290` params but only `20,490` of them trainable. Actually, that summarize the transfer learnin well.

In [32]:
# Compile the model
resnet_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [35]:
# Fit the resnet_model
resnet_history = resnet_model.fit(train_data,
                                  epochs=5,
                                  steps_per_epoch=len(train_data),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',
                                                                         experiment_name='resnet_v2_50')])

Saving TensorBoard log files to: tensorflow_hub/resnet_v2_50/20230621-123010
Epoch 1/5
24/24 [==============================] - 20s 845ms/step - loss: 0.7009 - accuracy: 0.7827 - val_loss: 0.7681 - val_accuracy: 0.7632
Epoch 2/5
24/24 [==============================] - 18s 789ms/step - loss: 0.5184 - accuracy: 0.8680 - val_loss: 0.7048 - val_accuracy: 0.7760
Epoch 3/5
24/24 [==============================] - 18s 780ms/step - loss: 0.4094 - accuracy: 0.9080 - val_loss: 0.6814 - val_accuracy: 0.7808
Epoch 4/5
24/24 [==============================] - 19s 804ms/step - loss: 0.3366 - accuracy: 0.9267 - val_loss: 0.6545 - val_accuracy: 0.7868
Epoch 5/5
24/24 [==============================] - 18s 785ms/step - loss: 0.2848 - accuracy: 0.9520 - val_loss: 0.6495 - val_accuracy: 0.7892


This transfer learning feature extractor model performed well. Compared with previous models, we get higher score and quicker training time by the way note that we only use 10 percent of the data.